# 10-year Yields through time

Equity investments are supposed to be for the "long-term." Usually, 10-years is where most investors peg their "long-term" to be. Investors have a choice between different asset classes - stocks/bonds/gold etc. With government/sovereign bonds, you know the yield (return) you can lock in, risk-free, for a decade today. Since equities are riskier, investors demand higher returns - the equity risk premium - over 10-year sovereign bond yields. This is why tracking the 10-year yield is important for equity investors - it denotes a floor for expected equity returns. Investors would be rational investors in equities only if they felt that it provides adequate excess returns over 10-year bonds.

Here, we plot the 10-year yields of US tresuries, Euro Area AAA rated soverigns and Indian Gsecs.

In [1]:
library(tidyverse)
library(ggthemes)
library(reshape2)
library(odbc)
library(plutoR)
library(quantmod)
library(lubridate)
library(ggrepel)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

#initialize
indices <- Indices() #for IndiaGsecTimeSeries
ycurves <- YieldCurve() #for UsTreasury, EuroArea 

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.0     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.1
✔ tidyr   0.8.3     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Attaching package: ‘reshape2’

The following object is masked from ‘package:tidyr’:

    smiths

Loading required package: xts
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Attaching package: ‘xts’

The following objects are masked from ‘package:dplyr’:

    first, last

Loading required package: TTR
Version 0.4-0 included new data defaults. See ?getSymbols.

Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date


Attaching package: ‘PerformanceAnalytics’

The following obje

In [2]:
startDate <- as.Date("2004-01-01")
# https://plutopy.readthedocs.io/en/latest/Indices.html#plutoPy.model.Indices.IndiaGsecTimeSeries
ycInd <- indices$IndiaGsecTimeSeries() %>%
    filter(NAME == "5_10" & TIME_STAMP >= startDate) %>%
    select(TIME_STAMP, YIELD = YTM) %>%
    collect() 

# https://plutopy.readthedocs.io/en/latest/YieldCurve.html#plutoPy.model.YieldCurve.UsTreasury
ycUst <- ycurves$UsTreasury() %>%
    filter(TIME_STAMP >= startDate) %>%
    select(TIME_STAMP, YIELD = Y10) %>%
    collect() 

# https://plutopy.readthedocs.io/en/latest/YieldCurve.html#plutoPy.model.YieldCurve.EuroArea
ycEuro <- ycurves$EuroArea() %>%
    filter(TENOR_Y == 10 & TENOR_M == 0 & CURVE_ID == 'G_N_A' & TIME_STAMP >= startDate) %>%
    select(TIME_STAMP, YIELD = VALUE) %>%
    collect() 

In [3]:
allYc <- ycInd %>% 
            outer_join(ycUst, by=c('TIME_STAMP')) %>% 
            outer_join(ycEuro, by=c('TIME_STAMP')) %>% 
            arrange(TIME_STAMP) %>%
            as.data.frame()

names(allYc) <- c('TIME_STAMP', 'INDIA', 'US', 'EURO')
print(head(allYc))
print(tail(allYc))

ERROR: Error in outer_join(., ycUst, by = c("TIME_STAMP")): could not find function "outer_join"


In [4]:
plotLines <- function(plotData){
    startDate <- first(plotData$TIME_STAMP)
    endDate <- last(plotData$TIME_STAMP)

    xAxisTicks <- seq(from=startDate, to=endDate, length.out=10)

    toPlot <- melt(plotData, id='TIME_STAMP')

    ggplot(toPlot, aes(x = TIME_STAMP, y=value, color=variable)) + 
        theme_economist() +
        geom_line() + 
        scale_x_date(breaks = xAxisTicks) +
        labs(x='', y='yield (%)', fill='', color='',
             title="10-year Sovereign Yields", 
             subtitle=sprintf("[%s:%s]", startDate, endDate)) +
        annotate("text", x=startDate, y=min(toPlot$value, na.rm=T), label = "@StockViz", 
                 hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)
}    

In [5]:
# plot the whole thing

plotLines(allYc)

# zoom in to the last 3 years

startNew <- as.Date(Sys.Date() - 3*365)
allYcNew <- allYc[allYc$TIME_STAMP >= startNew,]

plotLines(allYcNew)

ERROR: Error in first(plotData$TIME_STAMP): object 'allYc' not found


This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)